In [1]:
import numpy as np
import igl
import meshplot as mp

In [2]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [3]:
pi, v = igl.read_triangle_mesh("data/cat.off")
print(type(pi))
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 8})

<class 'numpy.ndarray'>


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# MLS function

In [4]:
# Creating the Aligned Bounding Box
min_p = pi.min(axis=0)
max_p = pi.max(axis=0)

bbox_min = min_p
bbox_max = max_p
bbox_diag = np.linalg.norm(bbox_max - bbox_min) 

In [5]:
# Generate grid n x n x n
n = 30
x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

In [6]:
pi_p = [] # list to hold the p+
pi_m = [] # list to hold the p-
eps = 0.01 * bbox_diag/2

for i in range(len(pi)):
    pi_p.append(pi[i] + eps * ni[i])
    pi_m.append(pi[i] - eps * ni[i])

# Conversion from list to numpy array
pi_p = np.array(pi_p)
pi_m = np.array(pi_m)

# Array of constraint points
c = np.concatenate((pi, pi_p, pi_m))

# List of constraint values
d = []
n = len(pi)
for i in range(3*n):
    if (i < n):
        d.append(0)
    elif (i < 2*n):
        d.append(eps)
    else:
        d.append(-eps)

# Conversion from list to numpy array
d = np.array(d)

# Ploting constraint points
p = mp.plot(pi, c='blue', shading={"point_size": 8})
p.add_points(pi_p, c='red', shading={"point_size": 8})
p.add_points(pi_m, c='green', shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

2

In [7]:
# Function to find closest point to constraint points
def find_closest_point(point, points):
    closest_index = -1
    i = 0
    closest_dist = float('inf')
    for p in points:
        if np.linalg.norm(p - point) < closest_dist:
            closest_dist = np.linalg.norm(p - point)
            closest_index = i
        i = i + 1       
    return closest_index

# Function to find closest constraint points within wendlandRadius h
def point_within_h(point, points, h):
    indexes = []
    for i in range(len(points)):
        if np.linalg.norm(point - points[i]) < h:
            indexes.append(i)
    return indexes

# Function to find closest constraint points within wendlandRadius h using Kd-Tree
def point_within_h_2(point, root, h, indexes = []): 
    # This is a helper function
    if (np.linalg.norm(point - root.point, 2) < h):
        if (root.index not in indexes):
            indexes.append(root.index)
    axis = root.depth % 3
    min_val = point[axis] - h
    max_val = point[axis] + h
    if (min_val < root.point[axis] and root.left != None):
        point_within_h_2(point, root.left, h, indexes)
    if (max_val >= root.point[axis] and root.right != None):
        point_within_h_2(point, root.right, h, indexes)
    return sorted(indexes)

In [8]:
# Building a Kd-Tree

class Node:
    def __init__(self, point, index, depth):
        self.left = None
        self.right = None
        self.point = point
        self.index = index
        self.depth = depth

class KdTree:
    def __init__(self):
        self.root = None
        self.nodes = []

    def insert(self, point, index):
        if self.root is None:
            self.root = Node(point, index, 0)
            self.nodes.append(self.root)
        else:
            self._insert(point, index, self.root, depth = 0)

    def _insert(self, point, index, cur_node, depth):
        axis = depth % 3
        if point[axis] < cur_node.point[axis]:
            if cur_node.left is None:
                cur_node.left = Node(point, index, depth + 1)
                self.nodes.append(cur_node.left)
            else:
                self._insert(point, index, cur_node.left, depth + 1)
        else:
            if cur_node.right is None:
                cur_node.right = Node(point, index, depth + 1)
                self.nodes.append(cur_node.right)
            else:
                self._insert(point, index, cur_node.right, depth + 1)
                
    def get_nodes(self):
        return self.nodes


In [9]:
# Function that return the basis polynomial functions
def basis_poly(z, k = 1):
    if k == 0:
        basis_func = np.array([1])
    elif k == 1:
        basis_func = np.array([1,z[0],z[1],z[2]])
    elif k == 2:
        basis_func = np.array([1,z[0],z[1],z[2],z[0]*z[1], z[0]*z[2], z[1]*z[2], z[0]**2, z[1]**2, z[2]**2])
    return basis_func

# Function that calculates the Wendland weight 
def weight(r,h):
    if (r < h):
        return ((1 - r/h)**4)*(4*(r/h) + 1)
    else:
        return 0

# Implicit MLS function
def f(x, c, d, h, k = 1, method = 1, Tree = None):
    if method == 1:
        indexes = point_within_h(x, c, h)
    if method == 2:
        indexes = point_within_h_2(x, Tree.root, h, [])
        
    n = len(indexes)
    coeffs = 3**k + 1 # Number of polynomial coefficients 
    
    if(n < 2*coeffs):
        return 100
    else:
        
        B = np.zeros([n,coeffs])
        W = np.eye(n)
        constraint_points = c[indexes]
        constraint_values = d[indexes]
        
        for i in range(n):
            B[i,:] = basis_poly(constraint_points[i],k)
            W[i,i] = weight(np.linalg.norm(x - constraint_points[i],2),h)

        A = ((B.T).dot(W)).dot(B)
        y = ((B.T).dot(W)).dot(constraint_values)
        ax = np.linalg.solve(A,y)
        return basis_poly(x,k).dot(ax) 

# Plot 1:

* Non-Accelerating Structure
* Polynomial Degree of 1
* Grid of n x n x n (n = 30)
* Wendland Radius = 14
* Time Taken to Execute: 2min 30s

In [11]:
%%time

# Parameters
method = 1
polyDegree = 1 # Degree of polynomial 
h = 14 # Wendland Radius

fx = np.array([f(xi, c, d, h, polyDegree, method) for xi in x])
ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1

CPU times: user 2min 30s, sys: 634 ms, total: 2min 31s
Wall time: 2min 30s


In [15]:
# Treshold fx to visualize inside outside
ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1
q = mp.plot(x, c=ind, shading={"point_size": 4,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Plot 2:

* Non-Accelerating Structure
* Polynomial Degree of 2
* Grid of n x n x n (n = 30)
* Wendland Radius = 22
* Time Taken to Execute: 2min 46s

In [10]:
%%time

# Parameters
method = 1
polyDegree = 2 # Degree of polynomial 
h = 20 # Wendland Radius

fx2 = np.array([f(xi, c, d, h, polyDegree, method) for xi in x])
ind = np.zeros_like(fx2)
ind[fx2 >= 0] = 1
ind[fx2 < 0] = -1

CPU times: user 4min 54s, sys: 2.73 s, total: 4min 57s
Wall time: 2min 46s


In [11]:
# Treshold fx to visualize inside outside
ind = np.zeros_like(fx2)
ind[fx2 >= 0] = 1
ind[fx2 < 0] = -1
q = mp.plot(x, c=ind, shading={"point_size": 4,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Plot 3:

* Accelerating Structure: Kd-Tree
* Polynomial Degree of 1
* Grid of n x n x n (n = 30)
* Wendland Radius = 14
* Time Taken to Execute: 24s
* Improvement of around 85%

In [12]:
# Creating the Tree
Tree = KdTree()

# Inserting all the nodes into Tree
for i in range(len(c)):
    Tree.insert(c[i], i)

In [13]:
%%time

# Parameters
method = 2
polyDegree = 1 # Degree of polynomial 
h = 14 # Wendland Radius
                
fx3 = np.array([f(xi, c, d, h, polyDegree, method, Tree) for xi in x])
ind3 = np.zeros_like(fx3)
ind[fx3 >= 0] = 1
ind[fx3 < 0] = -1

CPU times: user 23.6 s, sys: 97.2 ms, total: 23.7 s
Wall time: 23.8 s


In [14]:
# Treshold fx to visualize inside outside
ind = np.zeros_like(fx3)
ind[fx3 >= 0] = 1
ind[fx3 < 0] = -1
q = mp.plot(x, c=ind, shading={"point_size": 4,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Plot 4:

* Accelerating Structure: Kd-Tree
* Polynomial Degree of 1
* Grid of n x n x n (n = 30)
* Wendland Radius = 20
* Time Taken to Execute: 55.3s
* Improvement of around 76%

In [32]:
%%time

# Parameters
method = 2
polyDegree = 2 # Degree of polynomial 
h = 20 # Wendland Radius
                
fx4 = np.array([f(xi, c, d, h, polyDegree, method, Tree) for xi in x])
ind4 = np.zeros_like(fx3)
ind[fx4 >= 0] = 1
ind[fx4 < 0] = -1

CPU times: user 2min 38s, sys: 1.47 s, total: 2min 39s
Wall time: 55.3 s


In [33]:
# Treshold fx to visualize inside outside
ind = np.zeros_like(fx4)
ind[fx4 >= 0] = 1
ind[fx4 < 0] = -1
q = mp.plot(x, c=ind, shading={"point_size": 4,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Conclusions

* The data structure reduces the waiting time around 85% for a first degree polynomial
* The data structure reduces the waiting time around 76% for a second degree polynomial
* The results is better for a first degree polynomial
* The results is noissy for a second degree polynomial
* The radius depends on the distance of the points. The closer they are, the smaller the radius. For a fixed grid
* The radius also depend on the grid. The denser the grid, the smaller the radius. For a fixed radius
* The picture below show the reeconstruction of the object using a first degree polynomial.

# Marching to extract surface

In [15]:
# Marcing tet to extract surface
sv, sf, _, _ = igl.marching_tets(x, T, fx3, 0)
sf_cc_indexes = igl.face_components(sf)
sf_cc = sf[sf_cc_indexes == 0]
mp.plot(sv, sf_cc, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.2356796…

# Implicit Surface Reconstruction Using a non-axis-aligned grid.

# Reading Point Cloud

In [16]:
pi2, v2 = igl.read_triangle_mesh("data/luigi.off")
print(type(pi2))

pi2 /= 10
ni2 = igl.per_vertex_normals(pi2, v2)
mp.plot(pi2, shading={"point_size": 0.5, "width": 800, "height": 800})

<class 'numpy.ndarray'>


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.1287999…

# Transforming non-axis-aligned points to canonical axis-aligned points

In [17]:
# define a matrix that will hold all the data points
A = pi2.reshape((len(pi2),3))

# calculate the mean of each column
M = np.mean(A.T, axis=1)

# center columns by subtracting column means
C = A - M

# calculate covariance matrix of centered matrix
V = np.cov(C.T)

# eigendecomposition of covariance matrix
values, vectors = np.linalg.eig(V)

# Creating the Transformation Matrix 
transform_matrix = np.zeros((3,3))
transform_matrix[:,0] = vectors[0]
transform_matrix[:,1] = vectors[1]
transform_matrix[:,2] = vectors[2]

# Creating Rotation Matrix
theta = np.pi/2
Rot_Matrix = np.array([
    [np.cos(theta), np.sin(theta), 0],
    [-np.sin(theta), np.cos(theta), 0], 
    [0, 0, 1]
])

# Transforming the non-axis-aligned poinyts to canonical axis-aligned points
# This is similar as transforming the grid
# Rotating the data points to have a vertical shape
pi2_modified = np.array([Rot_Matrix.dot(transform_matrix.dot(pi2_i)) for pi2_i in pi2])
mp.plot(pi2_modified, shading={"point_size": 0.5})

ni2_modified = np.array([Rot_Matrix.dot(transform_matrix.dot(ni2_i)) for ni2_i in ni2])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.726429…

In [18]:
# Creating the Aligned Bounding Box
min_pi_2 = pi2_modified.min(axis=0)
max_pi_2 = pi2_modified.max(axis=0)

bbox_min_2 = min_pi_2
bbox_max_2 = max_pi_2
bbox_diag_2 = np.linalg.norm(bbox_max_2 - bbox_min_2)

In [19]:
pi2_p = [] # list to hold the p+
pi2_m = [] # list to hold the p-
eps2 = 0.01 * bbox_diag_2/2

for i in range(len(pi2_modified)):
    pi2_p.append(pi2_modified[i] + eps2 * ni2_modified[i])
    pi2_m.append(pi2_modified[i] - eps2 * ni2_modified[i])

# Conversion from list to numpy array
pi2_p = np.array(pi2_p)
pi2_m = np.array(pi2_m)

# Array of constraint points
c2 = np.concatenate((pi2_modified, pi2_p, pi2_m))

# List of constraint values
d2 = []
n2 = len(pi2_modified)
for i in range(3*n2):
    if (i < n):
        d2.append(0)
    elif (i < 2*n2):
        d2.append(eps2)
    else:
        d2.append(-eps2)

# Conversion from list to numpy array
d2 = np.array(d2)

# Ploting constraint points
p2 = mp.plot(pi2_modified, c='blue', shading={"point_size": 0.5})
p2.add_points(pi2_p, c='red', shading={"point_size": 0.5})
p2.add_points(pi2_m, c='green', shading={"point_size": 0.5})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.726429…

2

# Plot 5

* Using accelerating data structure
* Grid of n = 50
* Degree of Polynomial = 1
* Wendland Radius = .48
* Time taken: 46.2s

In [20]:
# Generate grid n x n x n
n2 = 50
x2, T2 = tet_grid((n2, n2, n2), bbox_min_2 - 0.05 * bbox_diag_2, bbox_max_2 + 0.05 * bbox_diag_2)

# Creating the Tree
Tree2 = KdTree()

# Inserting all the nodes into Tree
for i in range(len(c2)):
    Tree2.insert(c2[i], i)

In [21]:
%%time

# Parameters
method = 2
polyDegree = 1 # Degree of polynomial   
h2 = .48 # Wendland Radius

fx5 = np.array([f(xi, c2, d2, h2, polyDegree, method, Tree2) for xi in x2])
ind2 = np.zeros_like(fx5)
ind2[fx5 >= 0] = 1
ind2[fx5 < 0] = -1

q2 = mp.plot(x2, c=ind2, shading={"point_size": .5,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.726429…

CPU times: user 44.3 s, sys: 170 ms, total: 44.5 s
Wall time: 44.4 s


In [22]:
# Marching tet to extract surface
sv2, sf2, _, _ = igl.marching_tets(x2, T2, fx5, 0)
mp.plot(sv2, sf2, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.731333…

# Plot 6

* Using accelerating data structure
* Grid of n = 80
* Degree of Polynomial = 2
* Wendland Radius = .46
* Time taken: 3min 2s

In [71]:
# Generate grid n x n x n
n2 = 80
x2, T2 = tet_grid((n2, n2, n2), bbox_min_2 - 0.05 * bbox_diag_2, bbox_max_2 + 0.05 * bbox_diag_2)

In [74]:
%%time

# Parameters
method = 2
polyDegree = 2 # Degree of polynomial   
h2 = .46 # Wendland Radius

fx6 = np.array([f(xi, c2, d2, h2, polyDegree, method, Tree2) for xi in x2])
ind2 = np.zeros_like(fx6)
ind2[fx6 >= 0] = 1
ind2[fx6 < 0] = -1

q2 = mp.plot(x2, c=ind2, shading={"point_size": .3,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.726429…

CPU times: user 3min 1s, sys: 1.26 s, total: 3min 2s
Wall time: 3min 2s


In [80]:
# Marching tet to extract surface
sv2, sf2, _, _ = igl.marching_tets(x2, T2, fx6, 0)

sf2_cc_indexes = igl.face_components(sf2)
sf2_cc = sf2[sf2_cc_indexes > 1]
print(sf2_cc)
mp.plot(sv2, sf2_cc, shading={"wireframe": True})

[[    16     17     18]
 [    19     20     21]
 [    20     22     21]
 ...
 [125886 125885 125865]
 [125865 125866 125885]
 [125885 125866 125853]]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.726457…

# Conclusions

* Similar as above, the polynomial of degree one worked better with almost no noise and less grid points
* I increased the number of grid points since the object included more details
* The radius depends both on the distance between points and the also the number of grid points
* The technique to transform the object from a non-axis aligned system to an axis-aligned system was PCA
* A rotation matrix was used to make the object vertical